In [227]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from typing import Union, List, Tuple
import os

## Constants

In [228]:
M = 1 # kg
x0 = 2.8 # m
v0 = 0 # m/s
dt = 0.01
alpha = 0

## Helper Functions

In [229]:
def V(x: Union[float, np.ndarray]) -> Union[float, np.ndarray]:
    """
    Custom Potential Energy dependent on position
    """
    return -np.exp(-x**2)-1.2*np.exp(-(x-2)**2)  # J

In [230]:
def derivative(func, x0: float, n: int = 1) -> float:
    """
    Computes derivative of a function f where x=x0, using recurrency
    """
    dx = 1e-6
    if n == 1:
        dy = func(x0+dx) - func(x0)
        return dy/dx
    if n > 1:
        dy = derivative(func, x0 + dx, n-1) - derivative(func, x0, n-1)
        return dy / dx

In [231]:
def create_dir(path: str):
    """
    Creates directory if it doesn't exist already
    """
    if not os.path.exists(path):
        print(f"[INFO]::Creating a directory: {path}")
        os.makedirs(path)
        print(f"[SUCCESS]::directory creation completed!")

In [232]:
class PlotValue:
    def __init__(self, x: np.ndarray, y: np.ndarray, label: str = ""):
        self.x = x
        self.y = y
        self.label = label

## Plotter

## Solver

In [233]:

class Solver:
    def __init__(self, tmax: int, dt: float, alpha: float, mass: float):
        self.x = None
        self.v = None
        self.t = None
        self.alpha = None
        self.mass = None
        self.dt = None
        self.update_params(tmax, dt, alpha, mass)

    def update_params(self, tmax: int, dt: float, alpha: float, mass: float = M):
        num = int(tmax//dt)
        self.x = np.zeros(num)
        self.v = np.zeros(num)
        self.t = np.linspace(0, tmax, num=num)
        self.alpha = alpha
        self.mass = mass
        self.dt = dt

    def set_new_values(self, x: float, v: float, i):
        self.x[i] = x
        self.v[i] = v

    def get_new_values(self, i: int):
        return self.x[i], self.y[i]

    def calculate(self):
        for i in range(len(self.t)):
            if i == 0:
                self.set_new_values(x0, v0, 0)
                continue
            x, y = self.get_next_step(i-1)
            self.set_new_values(x, y, i)

## Euler Solver

In [234]:
class Euler(Solver):
    def __init__(self, tmax: int, dt: float, alpha: float, mass: float = M):
        super().__init__(tmax, dt, alpha, mass)

    def get_next_step(self, i: int):
        new_x = self.x[i] + self.v[i] * self.dt
        new_y = self.v[i] - 1 / self.mass * derivative(V, self.x[i]) * self.dt - self.alpha * self.v[i] * self.dt
        return new_x, new_y

## Trapezoid Solver

In [235]:
class Trapezoid(Solver):
    def __init__(self, tmax: int, dt: float, alpha: float, mass: float = M):
        super().__init__(tmax, dt, alpha, mass)

    def f1(self, xnext: float, vnext: float, i: int) -> float:
        return xnext - self.x[i] - self.dt / 2 * (vnext + self.v[i])

    def f2(self, xnext: float, vnext: float, i: int) -> float:
        return vnext - self.v[i] - self.dt / 2 * (-1 / self.mass * (derivative(V, xnext) - derivative(V, self.x[i])) - self.alpha * (vnext - self.v[i]))

    def get_b(self, xnext, vnext, i: int):
        return -np.array([self.f1(xnext, vnext, i), self.f2(xnext, vnext, i)])

    def get_A(self, xnext: float) -> np.ndarray:
        A = np.array([
            [1, -self.dt / 2],
            [self.dt / (2 * self.mass) * derivative(V, xnext, 2),
             1 + self.dt / 2 * self.alpha],
        ])
        return A

    def get_next_step(self, i: int) -> Tuple[float, float]:
        eps = 1e-3
        X = np.array([1., 1.])  # x_mu, v_mu

        b = self.get_b(X[0], X[1], i-1)
        while abs(b[0]) > eps and abs(b[1]) > eps:

            solve_x = np.linalg.solve(self.get_A(X[0]
            ), b)
            X = solve_x
            b = self.get_b(X[0], X[1], i-1)
        return X[0], X[1]




In [236]:
class Plotter:
    def __init__(self, solver: Union[Euler, Trapezoid], dir: str):
        self.solver = solver
        self.dir = dir

    def get_KE(self) -> np.ndarray:
        return 0.5 * M * self.solver.v ** 2

    def get_V(self):
        return V(self.solver.x)

    def get_TE(self):
        return self.get_KE() + self.get_V()

    def plot_values(self, filename: str = "funcT"):
        fig, axes = plt.subplots(3, 1, sharex=True)
        values = [
            PlotValue(self.solver.t, self.solver.x, "Position"),
            PlotValue(self.solver.t, self.solver.v, "Velocity"),
            PlotValue(self.solver.t, self.get_KE(), "Kinetic Energy"),
            PlotValue(self.solver.t, self.get_V(), "Potential Energy"),
            PlotValue(self.solver.t, self.get_TE(), "Total Energy"),
        ]
        for ax, v in zip(axes[:2], values[:2]):
            ax.plot(v.x, v.y)
            ax.set_title(v.label)
        ax3 = axes[2]
        ax3.set_title("Energy")
        for v in values[2:]:
            ax3.plot(v.x, v.y, label = v.label)
            ax3.legend()
        
        fig.tight_layout()
    
        create_dir(self.dir)


        fig.savefig(f"{self.dir}/{filename}.png")
        plt.close()

    def plot_phase(self, filename: str = "phase"):

        fig, ax = plt.subplots(1, 1)
        ax.plot(self.solver.x, self.solver.v)
        ax.set_title("v(x)")
        

        fig.savefig(f"{self.dir}/{filename}.png")
        plt.close()



## Handler

In [237]:
class Handler:
    def __init__(self):
        self.dir: str = "images"

    def exercise(self, S: type, dt: float, alpha: float, dir: os.path):
        tmax = 30
        solver: Union[Euler, Trapezoid] = S(tmax, dt, alpha)
        solver.calculate()
        mplt = Plotter(solver, os.path.join(self.dir, dir))
        mplt.plot_values()
        for tmax in [100, 1000]:
            solver.update_params(tmax, dt, alpha)
            solver.calculate()
            mplt.plot_phase(f"phase{tmax}dt{dt}")

    def exercise1(self, method: type = Euler, dir: str = "euler"):
        dt_arr = [0.01, 0.001]
        for dt in dt_arr:
            self.exercise(method, dt, 0, os.path.join(dir, "ex1"))

    def exercise2(self, method: type = Euler, dir: str = "euler"):
        dt = 0.01
        alphas = [0.5, 5, 201]
        for alpha in alphas:
            self.exercise(method, dt, alpha, os.path.join(dir, "ex2", f"alpha{alpha}"))

    def exercises(self, trapezoid: bool = False):
        if trapezoid:
            self.exercise1(Trapezoid, "trapezoid")
            self.exercise2(Trapezoid, "trapezoid")
        else:
            self.exercise1()
            self.exercise2()




    def run(self):
        # self.exercises()
        self.exercises(trapezoid=True)


In [238]:
h = Handler()

In [239]:
h.run()

[4.55161625e-17 7.79908984e-03]
[ 3.94107490e-17 -2.03797753e-02]
[1.07688381e-16 7.80102675e-03]
[-3.58599869e-16 -2.03816893e-02]
[-8.21554196e-17  7.80425543e-03]
[-2.37006595e-16 -2.03848759e-02]
[-3.99528501e-17  7.80877660e-03]
[ 7.26415456e-18 -2.03893301e-02]
[6.54858112e-17 7.81459129e-03]
[-2.01444764e-16 -2.03950448e-02]
[-6.66784336e-18  7.82170081e-03]
[ 3.17454396e-16 -2.04020108e-02]
[-1.55040911e-17  7.83010677e-03]
[-2.23128807e-16 -2.04102171e-02]
[-5.10659223e-17  7.83981105e-03]
[-2.84711490e-16 -2.04196505e-02]
[-6.54858112e-17  7.85081584e-03]
[ 3.49113100e-17 -2.04302959e-02]
[8.32667268e-17 7.86312359e-03]
[ 2.78639958e-16 -2.04421364e-02]
[8.43509290e-17 7.87673703e-03]
[-2.16840434e-16 -2.04551530e-02]
[1.02131845e-16 7.89165918e-03]
[-1.87783816e-16 -2.04693250e-02]
[6.54858112e-17 7.90789332e-03]
[-7.19910243e-17 -2.04846299e-02]
[-1.08420217e-18  7.92544299e-03]
[-3.68628739e-16 -2.05010434e-02]
[4.77048956e-17 7.94431201e-03]
[-1.64798730e-16 -2.05185394e-

KeyboardInterrupt: 